In [1]:
# 코드 9-27 라이브러리 호출
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import (
    train_test_split,
)  # 데이터셋을 훈련과 테스트 용도로 분리하기 위한 라이브러리
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
)  # 정규화와 관련된 라이브러리

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# 코드 9-28 데이터셋 경로 지정 및 훈련과 테스트 용도로 분리
df = pd.read_csv("../chap09/data/diabetes.csv")
X = df[df.columns[:-1]]  # 여덟 개의 칼럼은 당뇨병을 예측하는 데 사용
y = df["Outcome"]  # 당뇨병인지 아닌지 나타내는 레이블(정답)

X = X.values
y = torch.tensor(y.values)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [3]:
# 코드 9-29 훈련과 테스트용 데이터를 정규화
ms = MinMaxScaler()  # ①
ss = StandardScaler()  # ②

X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)
y_train = y_train.reshape(-1, 1)  # (?, 1)의 형태를 갖도록 변경, 즉 열의 수만 1로 고정
y_test = y_test.reshape(-1, 1)
y_train = ms.fit_transform(y_train)
y_test = ms.fit_transform(y_test)

In [4]:
# 코드 9-30 커스텀 데이터셋 생성
class customdataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.len = len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len

In [5]:
# 코드 9-31 데이터로더에 데이터 담기
train_data = customdataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data = customdataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

In [6]:
# 코드 9-32 네트워크 생성
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.layer_1 = nn.Linear(
            8, 64, bias=True
        )  # 칼럼이 여덟 개이므로 입력 크기는 8을 사용
        self.layer_2 = nn.Linear(64, 64, bias=True)
        self.layer_out = nn.Linear(
            64, 1, bias=True
        )  # 출력으로는 당뇨인지 아닌지를 나타내는 0과 1의 값만 가지므로 1을 사용
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        return x

In [7]:
# 코드 9-33 손실 함수와 옵티마이저 지정
epochs = 1000 + 1
print_epoch = 100
LEARNING_RATE = 1e-2

model = binaryClassification()
model.to(device)
print(model)
BCE = nn.BCEWithLogitsLoss()  # ①
optimizer = torch.optim.SGD(
    model.parameters(), lr=LEARNING_RATE
)  # 훈련 데이터셋에서 무작위로 샘플을 추출하고 그 샘플만 이용해서 기울기를 계산

binaryClassification(
  (layer_1): Linear(in_features=8, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [8]:
# 코드 9-34 모델 성능 측정 함수 정의
def accuracy(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (
        (y_pred_tag == y_test).sum().float()
    )  # 실제 정답과 모델의 결과가 일치하는 개수를 실수 형태로 변수에 저장
    acc = correct_results_sum / y_test.shape[0]
    acc = torch.round(acc * 100)  # ①
    return acc

In [13]:
# 코드 9-35 모델 학습
for epoch in range(epochs):
    iteration_loss = 0.0  # 변수를 0으로 초기화
    iteration_accuracy = 0.0

    model.train()  # 모델 학습
    for i, data in enumerate(
        train_loader
    ):  # 데이터로더에서 훈련 데이터셋을 배치 크기만큼 불러옵니다.
        X, y = data
        X, y = X.to(device), y.to(device)  # Move tensors to the same device as the model
        y_pred = model(X.float())  # 독립 변수를 모델에 적용하여 훈련
        loss = BCE(
            y_pred, y.reshape(-1, 1).float()
        )  # 모델에 적용하여 훈련시킨 결과와 정답(레이블)을 손실 함수에 적용

        iteration_loss += loss  # 오차 값을 변수에 누적하여 저장
        iteration_accuracy += accuracy(
            y_pred, y
        )  # 모델 성능(정확도)을 변수에 누적하여 저장
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % print_epoch == 0:
        print(
            "Train: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}".format(
                epoch, iteration_loss / (i + 1), iteration_accuracy / (i + 1)
            )
        )

    iteration_loss = 0.0
    iteration_accuracy = 0.0
    model.eval()  # 모델 검증(테스트)
    for i, data in enumerate(
        test_loader
    ):  # 데이터로더에서 테스트 데이터셋을 배치 크기만큼 불러옵니다.
        X, y = data
        X, y = X.to(device), y.to(device)  # Move tensors to the same device as the model
        y_pred = model(X.float())
        loss = BCE(y_pred, y.reshape(-1, 1).float())
        iteration_loss += loss
        iteration_accuracy += accuracy(y_pred, y)
    if epoch % print_epoch == 0:
        print(
            "Test: epoch: {0} - loss: {1:.5f}; acc: {2:.3f}".format(
                epoch, iteration_loss / (i + 1), iteration_accuracy / (i + 1)
            )
        )

Train: epoch: 0 - loss: 0.74742; acc: 49.222
Test: epoch: 0 - loss: 0.68719; acc: 56.250
Train: epoch: 100 - loss: 0.45943; acc: 78.556
Test: epoch: 100 - loss: 0.50736; acc: 75.500
Train: epoch: 200 - loss: 0.36649; acc: 86.444
Test: epoch: 200 - loss: 0.47231; acc: 75.250
Train: epoch: 300 - loss: 0.38391; acc: 83.667
Test: epoch: 300 - loss: 0.48900; acc: 75.500
Train: epoch: 400 - loss: 0.33234; acc: 85.556
Test: epoch: 400 - loss: 0.49467; acc: 74.500
Train: epoch: 500 - loss: 0.35625; acc: 86.222
Test: epoch: 500 - loss: 0.54631; acc: 74.750
Train: epoch: 600 - loss: 0.33831; acc: 88.333
Test: epoch: 600 - loss: 0.49766; acc: 75.750
Train: epoch: 700 - loss: 0.40947; acc: 83.444
Test: epoch: 700 - loss: 0.48575; acc: 75.000
Train: epoch: 800 - loss: 0.34388; acc: 87.444
Test: epoch: 800 - loss: 0.52711; acc: 74.250
Train: epoch: 900 - loss: 0.37066; acc: 80.333
Test: epoch: 900 - loss: 0.51784; acc: 76.500
Train: epoch: 1000 - loss: 0.36503; acc: 84.000
Test: epoch: 1000 - loss: 